# Functional Data Gathering - Areas

## Import modules

In [1]:
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from microns_phase3 import nda, utils

Connecting microns@db.datajoint.com:3306


## Building the dataframe of all functional cells

In [2]:
client = CAVEclient()

client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

#visualize available tables
client.materialize.get_tables()

['synapses_pni_2',
 'nucleus_neuron_svm',
 'nucleus_detection_v0',
 'functional_coreg',
 'allen_v1_column_types_slanted',
 'aibs_column_nonneuronal',
 'func_unit_em_match_release',
 'proofreading_status_public_release',
 'aibs_soma_nuc_metamodel_preds_v117']

In [3]:
df=client.materialize.query_table('functional_coreg') #dataframe of the functionally co-registered neurons
df.head() #visualize first couple neurons

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,pt_position
0,1,t,102531727972419182,864691136136642173,4,7,665,"[274864, 94064, 22046]"
1,2,t,94792540099826672,864691136208993724,4,7,740,"[218320, 104320, 19016]"
2,4,t,93946810036875732,864691135683934135,4,7,782,"[212128, 94544, 23423]"
3,5,t,84730565597057981,864691135334528233,4,7,855,"[145158, 110036, 19355]"
4,6,t,110905058841147285,864691136309730650,4,7,921,"[335744, 90128, 22805]"


This cell gives us the relevent identification values for each of the neurons in the co-registration subset so they can be used as inputs when gathering the activity data

All neurons measured in session 9 are removed due to issues when fetching the activity

In [4]:
session, scan_idx, unit_id, brain_area = (nda.AreaMembership & df).fetch('session','scan_idx','unit_id', 'brain_area')
df_areas = pd.DataFrame({'session': session, 'scan_idx': scan_idx, 'unit_id': unit_id, 'brain_area': brain_area})
df_areas = df_areas.loc[df_areas['session'] != 9]
df_areas

,session,scan_idx,unit_id,brain_area
0,4,7,648,RL
1,4,7,662,V1
2,4,7,664,V1
3,4,7,665,RL
4,4,7,671,V1
...,...,...,...,...
8190,8,5,10489,V1
8191,8,5,10542,V1
8192,8,5,10550,RL
8193,8,5,10567,RL


## Area V1

In [5]:
df_area_v1 = df_areas.loc[df_areas['brain_area'] == 'V1']
df_area_v1

,session,scan_idx,unit_id,brain_area
1,4,7,662,V1
2,4,7,664,V1
4,4,7,671,V1
5,4,7,682,V1
6,4,7,703,V1
...,...,...,...,...
8185,8,5,10458,V1
8186,8,5,10463,V1
8189,8,5,10480,V1
8190,8,5,10489,V1


This loop follows the same procedure as the one described in the layers data gathering file. You can look there for the explanation.

In [22]:
list_area_v1 = []
for row in range():
    unit_key = {'session':df_area_v1.iloc[row]['session'], 'scan_idx': df_area_v1.iloc[row]['scan_idx'], 'unit_id': df_area_v1.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_v1.append(avg_activity)
data_area_v1 = np.array(list_area_v1)
np.savetxt('v1_subsection.csv', data_area_v1, delimiter= ',')
    

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


## Area LM

In [6]:
df_area_lm = df_areas.loc[df_areas['brain_area'] == 'LM']
df_area_lm

,session,scan_idx,unit_id,brain_area
111,4,7,2711,LM
124,4,7,2763,LM
566,4,7,7716,LM
573,4,7,7748,LM
961,5,3,6766,LM
1291,5,6,1485,LM
1420,5,6,2390,LM
1432,5,6,2460,LM
1751,5,6,6752,LM
1854,5,6,7224,LM


In [55]:
list_area_lm = []
for row in range():
    unit_key = {'session': df_area_lm.iloc[row]['session'], 'scan_idx': df_area_lm.iloc[row]['scan_idx'], 'unit_id': df_area_lm.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_lm.append(avg_activity)
data_area_lm = np.array(list_area_lm)
np.savetxt('lm_subsection.csv', data_area_lm, delimiter= ',')

## Area AL

In [7]:
df_area_al = df_areas.loc[df_areas['brain_area'] == 'AL']
df_area_al

,session,scan_idx,unit_id,brain_area
64,4,7,2573,AL
65,4,7,2574,AL
66,4,7,2585,AL
68,4,7,2594,AL
69,4,7,2605,AL
...,...,...,...,...
8090,8,5,8749,AL
8093,8,5,9762,AL
8094,8,5,9774,AL
8152,8,5,10241,AL


In [ ]:
list_area_al = []
for row in range():
    unit_key = {'session': df_area_al.iloc[row]['session'], 'scan_idx': df_area_al.iloc[row]['scan_idx'], 'unit_id': df_area_al.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_al.append(avg_activity)
data_area_al = np.array(list_area_al)
np.savetxt('al_subsection.csv', data_area_al, delimiter= ',')

## Area RL

In [8]:
df_area_rl = df_areas.loc[df_areas['brain_area'] == 'RL']
df_area_rl

,session,scan_idx,unit_id,brain_area
0,4,7,648,RL
3,4,7,665,RL
29,4,7,921,RL
45,4,7,999,RL
53,4,7,1040,RL
...,...,...,...,...
8187,8,5,10464,RL
8188,8,5,10473,RL
8192,8,5,10550,RL
8193,8,5,10567,RL


In [ ]:
list_area_rl = []
for row in range():
    unit_key = {'session': df_area_rl.iloc[row]['session'], 'scan_idx': df_area_rl.iloc[row]['scan_idx'], 'unit_id': df_area_rl.iloc[row]['unit_id']}
    oracle_traces, score = utils.fetch_oracle_raster(unit_key)
    avg_activity = []
    for i in range(6):
        for j in range(63):
            avg_activity.append(sum(oracle_traces[:, i, j])/10)
    list_area_rl.append(avg_activity)
data_area_rl = np.array(list_area_rl)
np.savetxt('rl_subsection.csv', data_area_rl, delimiter= ',')